In [ ]:
# cooecnt to sqlalchedemy and create a session

from sqlalchemy.orm import sessionmaker
from src.service.models import Base, Annotator, Annotation, Dataset, Sample
from sqlalchemy import create_engine
import os

from dotenv import load_dotenv

load_dotenv("./vars.env")


# get engine from url
POSTGRES_URL = os.getenv("POSTGRES_URL")

engine = create_engine(POSTGRES_URL)
Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
# get all datasets 

datasets = session.query(Dataset).all()

In [ ]:
dataset = datasets[0]

In [ ]:
# get all samples  of dataset

samples = session.query(Sample).filter(Sample.dataset_id == dataset.id).filter(Sample.asr_text == None).all()


In [ ]:
samples

In [ ]:
# check all samples of dataset in which the asr_text is null and run a process to get the asr_text until there is no sample left
from src.utils.audio import asr_and_trim
from tqdm import tqdm

dataset = datasets[0]
language = dataset.language
samples = session.query(Sample).filter(Sample.dataset_id == dataset.id).filter(Sample.asr_text == None).all()
while len(samples) > 0:
    for sample in tqdm(samples):
        # get asr_text
        response = asr_and_trim(sample.s3url, language)
        sample.asr_text = response["asr_text"]
        sample.trim_start = response["trim_start"]
        sample.trim_end = response["trim_end"]
        sample.longest_pause = response["longest_pause"]
        
        # update sample
        session.add(sample)
        session.commit()
        
    # get samples with asr_text = null
    samples = session.query(Sample).filter(Sample.dataset_id == dataset.id).filter(Sample.asr_text == None).all()

In [ ]:
response

In [ ]:
0 * * * * /home/ubuntu/miniconda3/envs/corpus-insight/bin/python /home/ubuntu/corpus-insight/src/utils/trim_asr.py